<a href="https://colab.research.google.com/github/HanifAmeen/SE-44-Dermorax/blob/ML-Model---InceptionResNet-V2%2BSoft-Attention/SDGP_CNN_Model_IRV2%2BSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [ ]:
#mounting the drive 
from google.colab import drive
drive.mount('/content/drive')
print("done")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
done


In [58]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from glob import glob
import shutil

from sklearn.model_selection import train_test_split

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

Initializing the dataset

In [40]:
data_pd = pd.read_csv('/content/drive/MyDrive/Dataset/HAM10000_metadata')
data_pd.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern


In [41]:
def count_images_on_type(series):
    display(pd.DataFrame(series.value_counts()))

class_list = ['akiec','bcc','bkl','df','mel','nv','vasc']
count_images_on_type(data_pd['dx'])


,dx
nv,6705
mel,1113
bkl,1099
bcc,514
akiec,327
vasc,142
df,115


In [42]:
train_dir = os.path.join('HAM10000', 'train_dir')

test_dir = os.path.join('HAM10000', 'test_dir')


In [ ]:
#making the directories to store test and train data
os.mkdir('/content/drive/MyDrive/Dataset/train_dir')
os.mkdir('/content/drive/MyDrive/Dataset/test_dir')

In [43]:
df_count = data_pd.groupby('lesion_id').count()
df_count.head()

,image_id,dx,dx_type,age,sex,localization,dataset
lesion_id,,,,,,,
HAM_0000000,2,2,2,2,2,2,2
HAM_0000001,1,1,1,1,1,1,1
HAM_0000002,3,3,3,3,3,3,3
HAM_0000003,1,1,1,1,1,1,1
HAM_0000004,1,1,1,1,1,1,1


In [44]:
df_count = df_count[df_count['dx'] == 1]
df_count.reset_index(inplace=True)
df_count.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset
0,HAM_0000001,1,1,1,1,1,1,1
1,HAM_0000003,1,1,1,1,1,1,1
2,HAM_0000004,1,1,1,1,1,1,1
3,HAM_0000007,1,1,1,1,1,1,1
4,HAM_0000008,1,1,1,1,1,1,1


In [45]:
#finding the duplicates to make sure none of these images have augmented duplicates

def duplicates(x):
    unique = set(df_count['lesion_id'])
    if x in unique:
        return 'no' 
    else:
        return 'duplicate'

data_pd['is_duplicate'] = data_pd['lesion_id'].apply(duplicates)
data_pd.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,is_duplicate
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern,duplicate
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern,duplicate
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern,duplicate
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern,duplicate
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern,duplicate


In [46]:
df_count = data_pd[data_pd['is_duplicate'] == 'no']
print(df_count)

         lesion_id      image_id     dx  ... localization       dataset is_duplicate
10     HAM_0001396  ISIC_0025276    bkl  ...        trunk  vidir_modern           no
15     HAM_0007207  ISIC_0031326    bkl  ...         back  vidir_modern           no
20     HAM_0006071  ISIC_0032343    bkl  ...         face  vidir_modern           no
33     HAM_0005612  ISIC_0024981    bkl  ...        scalp  vidir_modern           no
34     HAM_0005388  ISIC_0027815    bkl  ...        chest     rosendahl           no
...            ...           ...    ...  ...          ...           ...          ...
9988   HAM_0001036  ISIC_0027588  akiec  ...         face     rosendahl           no
9990   HAM_0004462  ISIC_0027334  akiec  ...        trunk  vidir_modern           no
9991   HAM_0001152  ISIC_0030133  akiec  ...         face     rosendahl           no
10001  HAM_0000020  ISIC_0031922  akiec  ...         face     rosendahl           no
10008  HAM_0001576  ISIC_0033705  akiec  ...         face  vidir_

Test Train Split 15% to 85%

In [47]:
train, test_df = train_test_split(df_count, test_size=0.15, stratify=df_count['dx'])

In [48]:
#labels the data as test or train
def identify_trainOrtest(x):
    test_data = set(test_df['image_id'])
    if str(x) in test_data:
        return 'test'
    else:
        return 'train'


In [49]:
test_df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,is_duplicate
5599,HAM_0002162,ISIC_0030708,nv,follow_up,55.0,female,lower extremity,vidir_molemax,no
5206,HAM_0006254,ISIC_0024633,nv,follow_up,45.0,male,lower extremity,vidir_molemax,no
8437,HAM_0001111,ISIC_0026209,nv,histo,70.0,male,back,rosendahl,no
3343,HAM_0006416,ISIC_0028137,nv,follow_up,35.0,female,upper extremity,vidir_molemax,no
3380,HAM_0003092,ISIC_0027301,nv,follow_up,40.0,female,lower extremity,vidir_molemax,no


In [50]:
#creating train_df
data_pd['train_test_split'] = data_pd['image_id'].apply(identify_trainOrtest)
train_df = data_pd[data_pd['train_test_split'] == 'train']
train_df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,is_duplicate,train_test_split
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern,duplicate,train
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern,duplicate,train
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern,duplicate,train
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern,duplicate,train
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern,duplicate,train


In [51]:
# adding to lists by image id of train and test images
train_list = list(train_df['image_id'])
test_list = list(test_df['image_id'])

In [52]:
len(train_list)

9187

In [53]:
len(test_list)

828

In [54]:
# Set the image_id as the index in data_pd
data_pd.set_index('image_id', inplace=True)

In [55]:
targetnames = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

In [56]:
#making the directories per each cancer type in test and train dir 
for i in targetnames:
  os.mkdir("/content/drive/MyDrive/Dataset/test_dir/"+i)
  os.mkdir("/content/drive/MyDrive/Dataset/train_dir/"+i)

In [64]:
#compying images to train folders
for image in train_list:
    file_name = image+'.jpg'
    label = data_pd.loc[image, 'dx']

    # path of source image 
    source = os.path.join('/content/drive/MyDrive/Dataset/HAM10000/',file_name)

    # copying the image from the source to target file
    target = os.path.join('/content/drive/MyDrive/Dataset/train_dir/',label,file_name) 

    shutil.copyfile(source, target)

In [65]:
#copying images to the test folders
for image in test_list:

    file_name = image+'.jpg'
    label = data_pd.loc[image, 'dx']

    # path of source image 
    source = os.path.join('/content/drive/MyDrive/Dataset/HAM10000/',file_name)

    # copying the image from the source to target file
    target = os.path.join('/content/drive/MyDrive/Dataset/test_dir/',label,file_name)

    shutil.copyfile(source, target)

Model